In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Keys, ActionChains
import pandas as pd


In [2]:
driver = webdriver.Firefox(executable_path=r'/Users/rogerjunior/geckodriver')

/var/folders/41/t4024s9j7lsf7jdsjsqxx5280000gn/T/ipykernel_6711/2890382494.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=r'/Users/rogerjunior/geckodriver')


In [9]:
places_list = [
#     "https://www.google.com/maps/place/Drama+bar+Lisboa/@38.7213001,-9.1317669,15z/data=!4m2!3m1!1s0x0:0x6f20fd9f7f6c6e0f?sa=X&ved=2ahUKEwiQyu_b6e2BAxX2SKQEHe8WDh8Q_BJ6BAhAEAA&ved=2ahUKEwiQyu_b6e2BAxX2SKQEHe8WDh8Q_BJ6BAhTEAg",
#     "https://www.google.com/maps/place/Damas/@38.7164236,-9.129447,15z/data=!4m16!1m9!3m8!1s0xd19338983dc3d31:0x5be1f78dc6bc04c7!2sR.+da+Voz+do+Oper%C3%A1rio+60,+Lisboa!3b1!8m2!3d38.7165014!4d-9.1293831!10e5!16s%2Fg%2F11csd2bzzf!3m5!1s0xd19338983eac577:0x838e19df0563046c!8m2!3d38.7164888!4d-9.129402!16s%2Fg%2F11bwynq_mf?entry=ttu",
# "https://www.google.com/maps/place/Bar+of+Soap+-+Porto/@41.1514233,-8.607126,15z/data=!4m2!3m1!1s0x0:0x82402625ce48765?sa=X&ved=2ahUKEwjIz7GO7O2BAxUqU6QEHYaBC-QQ_BJ6BAhXEAA&ved=2ahUKEwjIz7GO7O2BAxUqU6QEHYaBC-QQ_BJ6BAhwEAg",
"https://www.google.com/maps/place/Pride+Bar/@41.1601423,-8.6055748,15z/data=!4m2!3m1!1s0x0:0x7f27efb4ca47b48c?sa=X&ved=2ahUKEwiavr3R7O2BAxVRUqQEHb2YAqQQ_BJ6BAhNEAA&ved=2ahUKEwiavr3R7O2BAxVRUqQEHb2YAqQQ_BJ6BAhuEAg",
# "https://www.google.com/maps/place/Cafe+Lusitano/@41.1491366,-8.6143825,15z/data=!4m6!3m5!1s0xd2464fd3396ad55:0xd760947441be9689!8m2!3d41.1491366!4d-8.6143825!16s%2Fg%2F1td1r4rt?entry=ttu",
# "https://www.google.com/maps/place/Prestige+Dance+Club/@37.0179259,-7.9369255,15z/data=!4m2!3m1!1s0x0:0x97c261e7c421ffe2?sa=X&ved=2ahUKEwjBhdWR7e2BAxVmT6QEHVb7ClAQ_BJ6BAhAEAA&ved=2ahUKEwjBhdWR7e2BAxVmT6QEHVb7ClAQ_BJ6BAhSEAg",
# "https://www.google.com/maps/place/Invictus+Caf%C3%A9+%2Fbar/@41.1501768,-8.6143183,15z/data=!4m2!3m1!1s0x0:0x801fb29e28bde262?sa=X&ved=2ahUKEwjewdan7e2BAxV1caQEHVNXArAQ_BJ6BAg_EAA&ved=2ahUKEwjewdan7e2BAxV1caQEHVNXArAQ_BJ6BAhhEAg",
# "https://www.google.com/maps/place/Glitz+Club+Leiria/@39.7729291,-8.7711848,15z/data=!4m2!3m1!1s0x0:0x3a552ac360bfb3ef?sa=X&ved=2ahUKEwjJxe_C7e2BAxVQTqQEHeL4BCkQ_BJ6BAhFEAA&ved=2ahUKEwjJxe_C7e2BAxVQTqQEHeL4BCkQ_BJ6BAhSEAg",
# "https://www.google.com/maps/place/Nosso+bar/@40.2817346,-7.5052758,15z/data=!4m5!3m4!1s0x0:0x15844d6c30f70ba6!8m2!3d40.2817346!4d-7.5052758?shorturl=1",
# "https://www.google.com/maps/place/Connection+Bar/@37.0929499,-8.2278511,17z/data=!3m1!4b1!4m6!3m5!1s0xd1ab595879b6fc1:0x566121ba6a73facd!8m2!3d37.0929499!4d-8.2278511!16s%2Fg%2F1td0c63t?hl=en-PT&entry=ttu",
# "https://www.google.com/maps/place/Jinky's+Bar/@37.0941247,-8.2285372,17z/data=!3m1!4b1!4m6!3m5!1s0xd1acea9e8bcc5a5:0x8a9235706001fdc1!8m2!3d37.0941247!4d-8.2285372!16s%2Fg%2F11bbrkqds4?hl=en-PT&entry=ttu",
# "https://www.google.com/maps/place/Dark+by+Nude/@37.0978611,-8.2276319,17z/data=!3m1!4b1!4m6!3m5!1s0xd1acfe37f055d3b:0xcef29f06ba074d21!8m2!3d37.0978611!4d-8.2276319!16s%2Fg%2F11td4pnwhx?hl=en-PT&entry=ttu",
# "https://www.google.com/maps/place/THE+FOREST+-+Evening+Lounge+%26+Night+Club/@37.0978785,-8.2281747,17z/data=!3m1!4b1!4m6!3m5!1s0xd1acfd93bc54f8b:0xfbe5e8deb6e4040f!8m2!3d37.0978785!4d-8.2281747!16s%2Fg%2F11stx7lxq_?hl=en-PT&entry=ttu",
# "https://www.google.com/maps/place/The+Loft/@37.1410457,-8.5380294,17z/data=!3m1!4b1!4m6!3m5!1s0xd1b288d729abaa3:0xf80893be5760bd35!8m2!3d37.1410457!4d-8.5380294!16s%2Fg%2F11bw4xcx0d?hl=en-PT&entry=ttu",
# "https://www.google.com/maps/place/Zoom+Porto/@41.1468197,-8.6082796,15z/data=!4m2!3m1!1s0x0:0x7e16e4adb8b22934?sa=X&ved=2ahUKEwiS2Zep8O2BAxV9dqQEHXF0DasQ_BJ6BAhFEAA&ved=2ahUKEwiS2Zep8O2BAxV9dqQEHXF0DasQ_BJ6BAhiEAg",
# "https://www.google.com/maps/place/Glitters+Gallery+Lisboa/@38.7274756,-9.1457013,15z/data=!4m6!3m5!1s0xd1933bcf44d9d5b:0x4ecc9cbf3bf12779!8m2!3d38.7274756!4d-9.1457013!16s%2Fg%2F11rv9f_dcy?entry=ttu",
]

In [20]:
def get_all_data(url):

    row = {}

    driver.get(url)

    if (driver.title == "Antes de continuar para o Google Maps"):
        driver.implicitly_wait(0.5)
        accept_button = driver.find_elements(by=By.XPATH, value="//*[contains(text(), 'Aceitar tudo')]")
        accept_button[1].click()
        driver.implicitly_wait(0.5)
    

    try:
        address = driver.find_element(by=By.XPATH, value="//button[contains(@data-tooltip, 'Copiar endereço')]")
        row["address"] = address.text
    except:
        back_button = driver.find_element(by=By.CLASS_NAME, value="hYBOP")
        back_button.click()
        address = driver.find_element(by=By.XPATH, value="//button[contains(@data-tooltip, 'Copiar endereço')]")
        row["address"] = address.text

    try:
        plus_code = driver.find_element(by=By.XPATH, value="//button[contains(@data-tooltip, 'Copiar Plus Code')]")
        text = plus_code.get_attribute("aria-label") if (plus_code.text != "") else plus_code.text
        row["region"] = text.split(" ")[-1].lower()
    except:
        row["region"] = ""

    try:
        website = driver.find_element(by=By.XPATH, value="//a[contains(@data-tooltip, 'Abrir Website')]")
        row["website"] = website.get_attribute("href")
    except:
        row["website"] = ""


    try:
        horarios = driver.find_element(by=By.XPATH, value="//img[contains(@src, '//www.gstatic.com/images/icons/material/system_gm/2x/schedule_gm_blue_24dp.png')]")
        horarios.click()
        table = driver.find_element(by=By.CLASS_NAME, value="eK4R0e")
        opening_hours = table.text
        opening_hours.split("\n")

        days_of_the_week = {
            "domingo":"sunday",
            "segunda-feira":"monday",
            "terça-feira":"tuesday",
            "quarta-feira":"wednesday",
            "quinta-feira":"thursday",
            "sexta-feira":"friday",
            "sábado":"saturday"
        }

        raw_opening_hours = opening_hours.split("\n")
        formated_opening_hours = {}
        for index, x in enumerate(raw_opening_hours):
            if (index % 2) == 0:
                hours_raw = raw_opening_hours[index+1]

                if (hours_raw == "Encerrado"):
                    hours = {
                        "opens": "00:00",
                        "closes": "00:00"
                    }
                else:
                    hours = {
                        "opens": hours_raw.split("–")[0],
                        "closes": hours_raw.split("–")[1]
                    }
                    
                formated_opening_hours[days_of_the_week[x]] = hours

        row["opening_hours"] = formated_opening_hours
    except:
        row["opening_hours"] = ""

    try:
        gallery = driver.find_element(by=By.XPATH, value="//button[contains(@jsaction, 'pane.heroHeaderImage.click')]")
        
        gallery.click()

        driver.implicitly_wait(1)

        photos = driver.find_elements(by=By.CLASS_NAME, value="Uf0tqf")
        photo_urls = []

        for photo in photos:
            ActionChains(driver).scroll(200,200,0,2000).perform() #.move_to_element(photo).perform()
            
            try:
                photo_urls.append(photo.get_attribute('outerHTML').split("&quot;")[1])
            except:
                pass
        row["photos"] = photo_urls
    except:
        row["photos"] = ""
    
    try:
        coordinates = driver.current_url.split("@")[1].split(",")
        coordinates = {
            "latitude": coordinates[0],
            "longitude": coordinates[1]
        }
        row["coordinates"] = coordinates
    except:
        row["coordinates"] = ""

    print(row)
    
    return row

df = pd.DataFrame()

for url in places_list:
    df.append(get_all_data(url), ignore_index=True)


{'address': 'R. do Bonjardim 1121, 4000-307 Porto', 'region': 'porto', 'website': 'http://pridebar.wixsite.com/pridebar-gls-porto', 'opening_hours': {'wednesday': {'opens': '00:00', 'closes': '00:00'}, 'thursday': {'opens': '00:00', 'closes': '00:00'}, 'friday': {'opens': '00:00', 'closes': '00:00'}, 'saturday': {'opens': '00:00', 'closes': '00:00'}, 'sunday': {'opens': '00:00', 'closes': '05:00'}, 'monday': {'opens': '00:00', 'closes': '00:00'}, 'tuesday': {'opens': '00:00', 'closes': '00:00'}}, 'photos': ['https://lh5.googleusercontent.com/p/AF1QipOdqA_tEb1z585y6Y9vzmbHTinyyI47nqxRUaLV=s906-k-no', 'https://lh5.googleusercontent.com/p/AF1QipOhTbOsoKNzZ1lEAHff-8zCraV48p21F_lqX4Zo=s1354-k-no'], 'coordinates': {'latitude': '41.1601167', 'longitude': '-8.6055044'}}


In [ ]:
https://www.google.com/maps/place/Drama+bar+Lisboa/@38.7212836,-9.1316787,15z/data=!4m15!1m8!3m7!1s0xd19335819f724e5:0x6f20fd9f7f6c6e0f!2sDrama+bar+Lisboa!8m2!3d38.7213001!4d-9.1317669!10e5!16s%2Fg%2F11kqxtxn7c!3m5!1s0xd19335819f724e5:0x6f20fd9f7f6c6e0f!8m2!3d38.7213001!4d-9.1317669!16s%2Fg%2F11kqxtxn7c?entry=ttu
https://www.google.com/maps/place/Drama+bar+Lisboa/@38.7212836,-9.1316787,3a,75y,90t/data=!3m8!1e2!3m6!1sAF1QipO6DVuuJD4CXA202CbmDzdgwp_zTE3l68kj3e7r!2e10!3e12!6shttps:%2F%2Flh5.googleusercontent.com%2Fp%2FAF1QipO6DVuuJD4CXA202CbmDzdgwp_zTE3l68kj3e7r%3Dw86-h148-k-no!7i828!8i1427!4m16!1m8!3m7!1s0xd19335819f724e5:0x6f20fd9f7f6c6e0f!2sDrama+bar+Lisboa!8m2!3d38.7213001!4d-9.1317669!10e5!16s%2Fg%2F11kqxtxn7c!3m6!1s0xd19335819f724e5:0x6f20fd9f7f6c6e0f!8m2!3d38.7213001!4d-9.1317669!10e5!16s%2Fg%2F11kqxtxn7c?entry=ttu